_Note: See halfway down the notebook for SCAndinavia 2._
# SCAndinavia (crypto)

Challenge file is provided in [scandinavia.elf](scandinavia.elf).

This was a Side-Channel-Attack based crypto challenge. You were given the binary, and if execute it remotely it generates random 256-bit values `A` and `N`, and gives you the encrypted flag and a corresponding power trace.

Right off the bat, there's already a couple things we can do:
1) Reverse the binary
2) Analyse the power trace

## Step 1: Reversing the binary

This was not very straightforward as it uses the AVR architecture which we weren't familiar with, so most of the time was spent learning the AVR instruction set. It wasn't even immediately clear what the encryption was (RSA? ElGamal? ECC?). Fortunately, the debug symbols were left in the binary, and before too long we had reduced to encryption to the following pseudo-code:

```py
# Calculate a * b % n, where a,b,n are all unsigned 256-bit integers
def multiply(a, b, n):
    result = 0
    for i in range(256):
        if (a >> i) & 1: # if i'th bit is set
            result += b
            if result >= n:
                result -= n
        b += b
        if b >= n:
            b -= n
    return result
    
def print_ciphertext():
    print(multiply(flag, A, N))
```

Roughly speaking, if we can figure out from the power trace when the first condition is satisfied, then we can completely leak `flag`, without having to learn anything about `A` or `N` (or even needing `c`!). So let's look at the power trace then!

## Step 2: Analysing the power trace

Let's run it remotely once (for the purpose of this write-up we use `query_0.txt` from `queries.tgz` from the discord). This gives us the ciphertext `c` as well as a b64-zlib-encoded power trace `pt`:

In [1]:
import base64, zlib

c = 0x51e45b2a7ca9de28fa92241f286145108e876c1b28acf5a38b7b7a3d274178c1
pt = 'eJzNe2d0U9e67e/3xnjj/bpj3BuwwTY9oaQQEpKQBAzG2Faz5CrJkntvsiUX3LDBYNwbxhQ3uUhykVyxDbgXuUBCCoHkQEIILUAgEJqx9b611t5bWzY554xx74/nXzmDo73XXusrc85vrrLS0tKysvLysuK8g0nhXg4f2P3H//lf//v//oeV3Zp1a+xWLv+v//zP/1pmvcLGZuUKa6vl8Af/vWrt25s/+PjzPVyv4MSjp9vOjRvHB7qb1VWnquqaWgwdHYZWnaaxsVGja25tbW1p1mnhT6drRn866o/93+h/tLS195wdHBkdPtveeKIwO/NA1pGC0orKyoqy4oL8/ILismPHSvOyEsJ8hC779nFcPSQyuVwq9vISy0MVqYeO5hxMjpKLnB0cnPgeUl9/P5nEG/5FdfhEc/+YcXKkz6BR19Y16PSd3d1d7W0tsCL8xvPGH+68MJle/Tp0PMppi53txs9dA2JV8RF+Hnyem5/yaE3XyPTM5GB3S0Ntbb2mtaOru6tDj36ta2kzdPScn/z+9nOTaf72+Ok47vt2Nm9/yvePUSmj/D35XFdJWFJ2YUl+Zpyf0NHefi/Xw8c/MEAulcjD0yp7f3g0P3dn4lS040Zr6w07hMHxSapIX3ceV+SnylP3jM3MTpzv1Klrauq1bR30kmE39Yb2doOhTddwuuSQKtibt8/B0UXo5ePrJ/cRi2Wh+8s7v3nwev7BxcZkwfs2Vms/5vrFJibEBHjyXXhewQmHCksLDyoD3fbZ73LguEv9AgN8pRKfkKTSjm8eLpgeXdKmuMKv1nzE8YtNSowJ9OS58MURB463njfOTI306WEL1Y3Nhs7urk5Da7MWrafNAH/6Vq36ZOEBhb87Z6/DPq6bWObnJ5d6e3rLQlTZx7W9I5MTI/0duvq6ugZtW3tXVyf6Gp2upVXf3n12/NvfnplMpj+/acn0/mSdzdoP90kjlKqYILErhyOSR6cdLS4+mhYtc3W0372P7+njHwBHK5FHZJzs//GpyfRwtk7F2bLSev2nggBFoirKz4PrzPUKTS1t6h01Gt/0VrTmzt6h2Z/uz5lMT650HvX78m2bVe/t9gyNVylCfdz4ruLwjIqW88YL06N9bQ011TUNzYaunm74ZJ1WoyVH0K5v0dQcy9kf4ePq5LDXWeAhlfv5+qBo9ItMya82nBubHB/oaWuqq1U3wYZ1dUFGNJMD7Owdvnjtj3n44u8Nh2Wfr7dZ/b6Ddxi8O8TH3U0andd26ZHJ9OirepXLJqtla7cLAuOTE6P9PXgcoW98bl3P2Ozs5ECntq66uq6JhGMbHY6wLBQZp4qz4gM9uY4O+zgib5kvPgkvn+D4g8eaeoYnJ0fPdjbXo2BuaUfr0sO6dCgV0Enc/AtO4uFXmjSPj9fYrNvm5BNJToLL94Yt1fSNTU+N9rdrSSbBy2FL8VfBz7v6R7++8QQd5HdtByWfrbNZs9VREq5UxQZLhByum19Mem5xcU5KhITnYL/bSeAlD4CDFHt5y0ITck62nh03Tg736mG/yJMhwNBhNaMkOzMw9cPdl/Dku5PVSt4Hq2zf/pTnF02SjCMQR2Qc0/WPTxupFG80pziOMLIrzY1VpZAvXrArjhySL2hXpEGKzLKG7qGJydFzXc0NdSjFya5QewqfNXbpV4gz019XOo7Iv9hgu+YDOK04VWyIVMQX+SYeP3sNlvb4opqOwkAShTyeR1i2Zubua9iPlnS3rbZWq7c5y2MSE2ODvAWoNCQfLirJOxArFzjs3LWX5ykLwKXBNyqz6vxPf77+63p/UcCXa61Wbrb3ClclxYfJ3EWSmHzDd3+aTH/M1ir2vb182bpPhcHK/YnwMoGbf2rV6K150/yN/jzZdru3Vm5xkEQlJseHSgQuHDc/RWZ+afGR5DAvl927djsLxb6w9z4SaUBcrsZ465Xprx87D0u2r1pu+/5eaVRiUlyIxJXD8wxNK9OdnZydHu03aOpqqNL57+S9WEy+4qlp7vb4SShwK1a8zRQ4LsdVpjhS1TE8BVW1B6pqjRoaRme3ORD1+MktTTXlsGKJYB9KL0+pL0kviX90WmFt+/nxybGB7tZGlF4tdHrhdRk6+0a//gV2yfT8p548/51v265GqR2HUlvE5Yh8446cNgxMTk8O9ZCfs7MThYuhHV6uqa04mkLlNh/nNoSqJ+T2/vwqPcrtwTMot+sam1GsMi/Xd/aNfPUzevnLa72FQbs32q1+194jRIFfznERSCMPHNP2jU5ODPe1Q11SN+raINpQXWqGL4dq2Nk7OPPj/VcQbT/25AfuesdmFfp9nCouTObGF3iFpJRo+idmZ8bPtsOR1C46En17V9/IxeuotDz+ru2QFOXgB3vF4fEoWKG0xOYbvn0M4XOhLt5po9XytZ+6BkFpifJz57oIJBEpOcWleRkxMj7EoyMf0jPQn8Rj9cC1pwsvfx0oDdm13mrFxp0eYaokZbjMzdUjOLNh6neTae5a92HvbTZv2bzvKItJSooLFvOdWUHnzbEIOn9FTtPkb69Mz6+dyZV/tnq5zbt7xJEJSXGhUlcXjntgUlFj79j09Pi5Dq26prZe20oKN6mddImrx0HngYMOlThfqtkosyu0Z4YnJoahRtXXsss+2R/U7XEleTBbnyzctsZ2w3aOPEqpjArwgv7oEZRUoO4amjSO4VpQBxXSgE6HDkyoRIPTP96H7TXdGTul4Ly/ym7jDgG0e4wW+J5hhxqnbr+Cst6W6bnNzmrVVidZNMp3KJ8CcWgS5Hv+AYWv695dkO8edL5HHjh97sfHc89+PlcctHOd1cpNuzzDyf7yBN5hGRWtA9MXZ3Afqqpp0Bm6ukkfIt+D+lArilXch/aiWJWY+9D+/Go9ALOJoTN6qjAydRX9mnwN9MAXP58rCXXYZGu3+Uu3IIUqLlzuzue7ByQUIBgyQ8EQNcE++CS0zEk0Aww5qAzy4rHLqtebyip7K6m2j5vNgwuNKW4fQbP5yFkWpVRFB0J15HuHpZVpoaZPodqjRoHexmo2KND7Ic8glk1Pf+g4YtFDpUIuzyMwqaC+Z3R6euJ8F2l1rew4oNfeWF12ODFUzEcQytWTQCioMAExGcXqzsGJybHzqMLU4UZJ6pM5yXGFeXa166jvlxvg5Xu86JdzhD6RKTlFxbkZsb4IsjjyAPUF+FOor+/Knyj46pScLSus130iCIxLSsCQhS+OzKxsPT85Y6RglrkuabXmNgaRX5QZF8BEPgOzlIcqNKzIp/NG30LtN0T+ZRz5D/F+r7Xd8LGLLBJFvrcrzy0wrWrol+eQkr35fp+vtbLZsts7QgUp6SPi8T0DlVkFJYWHVIAad+3cw3GTAoj19ZEgrNn17YO5l7cnTsc6bbK2Xv8Z1BP8OTyem39CQX3v+CyA2A6AKmYQC1+DcXdX3/AFDIKe/IDbKrOHoVIRj+fuF5uRV1x0ZD8Ujz27djuR4iFDHeuoxvjby4WnVzqyxdCx7N53RB1LESIRuYqj8vSoPT79VpPE22K9bM3HvIC45ESEfQXi2KKuKxBuTy41JnA3Wy9fu52Pth5gFaBFzyAV4OKi7IRgD+fduxxc3CR+AfCFYmmgsqBl9t686Rnpj1b4bQmJimCxwNlFJI9JzyspOZoaKUWIZh8gGgJNfaOyqgevPzct3MPoHprf5yJofsoIXzce3zs63/DNH6YFFAMum1dY0THg7yXyCslUT96Bt13RZ7i9v+Itu63OvorkJEWQF6AcafShU+3D07PGoZ6W+poaIFnti5kIITJzN0cqo53ftbV9ZwcmMpGEyMRjIjMLRKa5vqZ68c8xqYCWyyTE3r9LiBbLloshlgGg3yUM/R4Dhhd/ijC8owRh+GAJIEfP4P3FjWcAOY6dAzCOq3IHq6ZDMgMY/8dDiIUX1/oKg+yh6W3Z5U41PS4Xd+z2QeP0xGAP0/C7qbcjuAC1ZOwbXEseYwqx1oJCQFVPLoRiMAWv79SxioGe5gIGGh9mJ4QQPuXqRX+7NDA2owS+fRwVspYG+ts7O5hChvDhDYwPcSXaYIPxIW65Qg4uBkeKivMOKPxE+3bjYuCHi4EsLOVYz/eP0Zq1KcIPKM6FmRrPmQNrLqrvGTEaR89S/KWVwgkEbP9z/iLwDIg7kFdcmJ0Y4gkBvcdFJAGiB73XJzixWH/x3stXv8+qE3jvrrRau53nr4A0gBSBfY4/WtuN2UWXTo3ZRXsXnbI6y5TtzPFFKfselbK4TMCSG84wnduCtqKMN8DPgfYswEZd7cpdtGRXz+ADdRN3Fkyvf+nLlX2yapnNuwi/YiTqwvUI3o9gz8z02Nl2AgqoAKLKOQaMcIK4FXkShE94DzDi8LTjZy4/WpiHjo8+2Xod+uREyHzgtjQjPqQKcneCzIfahjNf4hOUUKT/6v4CtBdDltdHrF7uLeC5U7DHdGugOOALUi3DVYkIwDg7C3xiDp1oxV23V68BiNdEIUQa4hkwRHwM+/DgIqFZaxmaJeS5ikMSDxUW52fG+Qsdd+1yIJKBL2oe6Sf6fng09/zXofKw3RusVrzzpXuoEoqKHIqKV0hqefP5qYuzY/36RsAJ9Tp9V08PXW2pfteqrTuel4Ykkr17nfjuCCfIJIBpfcOTjp5q7QdQOtRLNBItBqVsnDBD44TSMIITRAQneAg9gtKrR397bXp9oz9f/umqZSu37JFEYuKB0FxQcnFT3zg6uA7tIvWF0HCAIFd/n0NVa6gict9mW9uNnwsDFQCokBIilMUePmUYNM5MDp1pbaxFRc/A4matVNVCPDw5XCpwWkQUolILagyYKPS0Eh5OY3VStRiigDJX/vkGGkOgzOVy3QNUebWdw8apcYIh1IQut+upeNaTsmEm245vINsTmGw3QD/GRYf6tQ6T7Ynvbj3HNav5AJY9tv1r2QMVpKD4gxXa3hHj5AiDcQk0ojFu9znqyX9+i4ux7bpt+yQR8Ur0ZJ7IL6nyLBCzl78OliE4v3ITwHklhpt8gJvpFS0DU2a4ScseFO6htru2IjclUiZkh5EYNeam8Rt/vXp02ZAt+WQ1tMq9uFWGSIRcvlcwNNiSgqx4fxEOancS1HBIgYqscs0ZdqFrIxiVYmMdgBO/wfT7PsYtqwlOVCGc6MpzlUZlnWgbMM5ODZ9pBSYJdIwFkCnQZAmQLYQahsz1tDVRkAv9uo0RaoYu4L70/B+9BYGkL9FkjMcVyaJSc4qLc+GM+Ht32cMZyRDOo/r/tb9Mr24OlYfaI8r0pUcohvTAPzEwOsPAalLLlqh7LGhKdWJfqhOnU514gEBTOiNohoIJ4CMTaqXAP+032lL8UQFLFoh8Eyv6rwFGeDRbG+dMIbY4zMo5gILQIZUWAgrydNlt7wBNw5dGQXm6mbvzppfXe/N8d6yxsn3XQRwB9Q5qM9fVN6ny3LVnplc3zhcHfsnQl/gwmYgnEIcfON42OH1heqS3taG6qpYlo9Hf2o54PpLRpEtPh5bRBomMBlS7kwoNkr74dF6bW+BqdgtEDAKpFzOoJNZCSdRSpYcuiYjmt1rEsoe5JEbsz6tqO4tpvr5JzZKkmklJ7OylWuFT/G7UCnd7hdKtENH0pj6aptPgl0ViDSwS6+7y94pp+yLtkiaxk5cR0lu4a6xR8T9YZbPhE54vksMCvIQin7jirh8gXx5dVCtdNlkvR/gSg3J34BiSqIOQLkj7ONPWSNQLi+jpMCOpS7oMz+3s3iRAGl5sRm5xUU4KQbxOFOJFGl5IwpGTLWfHjJPDfQYtI8K9qSo9+lqX7rl9LSUuKvGTCfPupootZt4E7+gJ3mmlOjzWvIMsNDzo8GGpx7q/g5h//G1zmtuHdtZrPnLxjQEkFegFSMo9MKkQPRk4fSf7yaS5tdAtCCkuP/UWBu+GjAHwGYySHFiwwAuwK9XB2jW1+ChZYdSCeew3N2G/F+5Nq5OF21bbrP/YRc7UJ3FoYjbp5iJH+117SeGTSxCuVGQdMxe+vynjSBdfuDtVmyDYCqe8nUudsoAnlMVknzIMTc0aqcKH8DyTWggTE4V6iTKASnVgfK5m8ubzecibw1JSqiWRCYlE9fMIiMssKCk6vAg6eiPtK7WwrmNwEg160DBEbdmN2xALIP306SJKFyJ1E7gHpdeM3Zo3LfyGQNOa5UjXjEiAQiEVurigkUdd9wibrzNzFl0zQ0/eIMpJvNHnaJGI+eJ6Hyavtpi8Yjjm4uLqE5l6tLgkLz1GLthrb+9IijSrkY6SRqquRZWYPSPQd/T8dxvpBaqRQuFr7wY8xswPKN0G4bFogsd4dCOV+MceVo9cfzr359XuXPkO+Jx394gxFwdQIvAOR4LQFBpMQFWrrmX3Oy2RVcYu/frErOGvXMvS8KEgp5frzk7MTI32GZrM+iFbVhn9+gY6wkcUk1rz4T5E5JCGT6syCIdjRbiWgXMUQ9BTde1EQUasnxuqazxU19ApeSKZ/3Clrm900gK4sDhNe/d54+U7WOYndc327U/YMj+sXds/PmUc6TdYqFnM2vsoSYhRZWh6gomYLDqjsPLU8fzUcC8X+532SFIIDITA9vCUBisPVzb3j5LihVam+7uV3Z9RJwk/XM2WDTk8KPfAbzHB7MAEU8sULx0dve2EXCOC6QgtnRrYoQEEIpj1ZnJdt0QpY1r63C8Av/dutlu1ZSeW6bAkLI7Obf3q/vz8w4sNSfz3VqKBnz/QyGioEnx3/4T8+t6JCxfIvAj3wJ6ebjIvamZPRhDz5b+B+VILI8xXsxhrIOaLguXBxabF8yKewDtYdbCguOCgMoA9bKRlo4evX901VgMIWWG1/lNBEFZAPIXugWm4SLz+pfeo9GPbt1a+u1canZQMlUngwvUMTStvPmecpWMfR18XC52Sjg5JlRolFy4hOcm5p6GjGxmSg8+4gwkfRHJ+egBoYu7GQHmE42Zbu01fUCTHnQfRA6yu7fwEdM5eAAR1lAbBgJF/GX1v1gRRIwiKs2gEZsWbnlD/O51zZEnnbFscfOVL8KSYyAG/E30LOoH1qg8cpZFY33LlivxTTg398gJIX3+B3441y7EmCAV7cR1qw2dB1yHqLDrotjh/b6ouyfVDaIuQMZDL0YFiN3e/xIqz156b5m8NlQV9uWaZ9cadnuEJycpwH9gpkW9sRl5pcc7+cG/O7p27qTT918ofHow9/aYpibuZUf6wuoc4fnZRKdYvnewJx0fkA319SfvXDxYwbhBttbFavc3FNxY4fqAX35nrERCfVVBadDgpBNAw7oMYDUug3eTppu+8Bk7Qc1T26Wor2/cc6O7JEwWkVY/dgqP6few4IukrN+70AJIeHy4TujjxJZGZFTqKZ2vJ8KejkygDzVjHIpkOqPLh17oML5aOBdWXL0Itv314+gJwnZb6arr2W5LhfypjWVaZViIAUoAByDA9lL7cbqnHIkTrQQSlw0mhaIK4hx7meHstQgV1b0AFdGIsHUm5icSROS1fPYSD+06X4vreimWrP+L4K5DxwMvVVRpX0nP1OSIrNbGO7yxfvv4zYbAyOSHK143rQo0QiRzawCgD7RZ6ZhuK+iNJfzdCHECWFaIMNLKUAaKuIXiPpnjX+4ssMCEWJOWKI6cNg0bzCJElh9IpZ2jVqk/kp0f7Qnt3YNq7l6e3PCwx52RLP54BUmFAGhklS+iRp+Ty7ReMVL/GFiClDLoNGo24cD0CE/PrOtGQipTleubtZFyOfj915d4c/P4e6qNbV5NxuTI+EkleXE8oGMzvG9XmftPCvH9g6uo96HbP/tFXFLJno63d5p1u6OOhEgoEyAagJbzK0IQAsaXUBN/egTCA+p9gAMRthiluY+60BHNh1AsE5PWdyWqK27BQryQs6XAhElOhiuIhGuWn8Y3IOHn26p/zL24OHVskjiEhJ7GwoXd8Zmb8XAfFxixQi7nBApnLsQx9N1fPkKx6411oC9d7cqTb7ZbZvEeNyMXoLEJSSt8IiFoYomfQsw0ZjEYkFsvD0yt7f3i8MP/7TK0SWQfWfcIPUKA+6IGEyZCEbGLVcaeLVgBSTDzF/tHpxfVd5PyaG2g5m6UsmQXpKx05jGEhnhgWhH9rWHDnOnE9gbg29owgi0iHroGRnQ1mvQ4p9Q8gth4vsq6E+ni4yxRFHd9DFf7deCrKYQMk7A5RiCo5IRISluuGDwFR4nMsSmwOGzTQg0NYwKaYbJ8d8GRYdTgmDyIAE6GAPJEXAW909dKNpp0vRVlkOObINW80cgpdefT65a3xk9H7Nq5YsWGHMIgyW3FcpWjuDfwgWsZ32IX5QUAg5gc+gAqJdQzylDrcDjalRXly5R4Qx/lbaBj8np3tO5/hYTCauHAEqNo3nwW8Otyn19QRRmyJV+mhydPLhmwpfDFdGLFfxh2leffI1PTEAD5mtYVbp5U2SWhqifbJqnBAiYJUBS0zd+ZMz37qzvGBHmX3HmZ4CqSmuwjl0em5JSW5aVE+/L14biWjtEX45OzK5r4xmhKxITpNiSa/x6Xp0ddAL7avtV3/kTPCIpjtugWknBq8/gwQw9lCrIvT1hXSQeLxFDEhCAJ65x4XNEUE+C2RhSSXdVy6/+rlnckqBYGDrkFkKMrnITdAU//khQsTdOpaiDjMTPSN+RUB+XX5jzk49lMxTpvg2Cmxyw977JRAOEdnyKyDNdym2XfnGThcNC99/dtIZbTTFotxGjb2HSbGPtE+ythHTXVSK/C04ffpmjjnzRjc0iNeFz72AJQWYDsglC/GDkh+9ZgMBTlb0FCQT4aC8CuBNPrgSf2gccZIieCNzUu6DQ6E47mpSMly+HslSwPAlVbRWhncO0AUeNM9C3MV6hZ8DqJ8pZresampRTIFw20gDt9c4Zaq4OYxFjWVxrWK6E0oB3w+32C7dutecVi8EuUATyhPKO+98sQ0d2u0MtLhHesV73zhFoLnqJ5u4ogjuosPoA593ZjA2WS1bM12PrLJxWCbnDwOD7IuGAfRIMvicJH0h0w2RPJhBhqbkPEhIR4ZH3juAQn5lsYHKFU9i2alSIsqZk2baD+ZD8D58kbK+NBi8cnU5I5RSZ5ctih0scS5EJSEh5XT4+eIPExwmvnnBJpAnXxBa9JmgxDLexAjB55h78jzZPEMltyxhH4T2EFymyK5GLIj2AGtFwdCmaYPOTLOvsH+R1odTkViT3CkjDlE04RAOAQkZxi4v0VjYTFsurD88VVTqvvHxA4SSWMeYh4sykmN8hHsxcKjjBq1R2ZWDVx7hnpOVSxOcqjtRNHm8b2jctsuPVwAGFWfwIWsWrsdN1j4HpFncEbdxG0A8Vfbszy2rnzL9gMneWxyMhAfoUimOoZF8jsjZUFfrHnLauNOr4iEZFW4zE3gGXpIMwtR9/xKW4YITeg/dEETesQYILsjs07oB6eWziKaWSN2ip/K3shPW1GeorGhpbxBUNkZqtss3Jk4Hc99H+Up3y9GpYQ8dRVKYws7vn+CrQ/J/HdXLF/zMRfh6Gg0TAbEqDyI7A2JFlQGnYtvREpBTfvAhJE9WjePyFC0odEfsOKF22MnY11Qj9shCCCGJ4RSgpRZ+SWF2Umh3hwHewcXoVjuj0kSmgZN3pozvbpxrgiNBWy20K1AyHWFDe778alp7rfhY2FkPILBWrjcDRk/FAfySjDtcrbfudsZyY+oTaCBbNuFuy9f/X6B6BzUF8YgXOiGHNNnJi5cMBKdAzK2s7un28xBDCwHKBJgFjumof+8ujddp+Ki+fAnPBwl/ngkrsSG2wvUSFy9SJfE/Uf/b3CuLrZtwkIwxQIKcsxiarSVQQBcnmcwGqiPTk8zshJ7UkQfzhski/hwXw+BSJ5wrO8fEMZ/zFTH7nvHygrRJ4x63DjOPO+QRODEWJxxQv3LXYpdCW+Al2rKKWIwqz7/pr3yTBsWSpotrCJ6avYB/CA9xpfmB5T06RdzqHbo5+em+bsTJxnjTFwiMs5wnJEP/ZiOwmM0fSaDdR3xTjLmxz8uERcKMiAT8VLEc6X0qCyzHkXZmMJSKnq+ezj34tYYvPQdsxffz1PkFXaoaeY+IOnLLWmu71m/tWob0FOU78gXL4k6eBJL8IiCmh05ON/xMAXrx8jZfm+qLhEp+OvNCj6Xj9dTUnCQsR5IKOtBcGKx4ev78wuPLmlTkTGEkiUwJ5bEFnb+8BcSOhoTzTamxP+PbEzIbOSvylefGZ+dnWQuMFigNkuTa8Cut6keloDnqnx3MmXCxlHa+4UVoMD4PO3UrRfzj75rPeCxzXY58mTEJCYqgryhPxEnR8HBRSNmcrzfP5qfu2usUjhtsrZiXGoIBpL8nmEM9ZDfTOFg+QTfJJCK/WIP14/dhPR7OFOt2LfRivVkjNoOndQPII3AArWZR20QHeTewx9fadKojofUU9TxKE0Z4a6xJeMhCv7T7svFA7FFpvZOlvvSbPXvot2XTyivPjJsQeVBhi3gHgEJebWdQ8YpStMhNwUsBW0D0RSTGE2RUVcCotOL6jqgqWB1RW05o6alputIEXr189nikD2b7Fa9u8s9WIFn1MiJHJlRruklHmktNf/spPKcAo1QgSgDDRZEWQYavtAn+iC0YcoSUFNd19hi0YbxlR2clq9vT1TFA+C1QYAXX9mBViKSKw6fbh8i3nTiszNY+OxI7AJefn6tvzhk90Ybu8073YMhdpFODD+PSsW+jSgpopMomQidZG45TKFbDhr2hLSZwV9GPNadx+t6nwBx3OC9AJTEl/ZcfYat/3FOdKwlk7EuE2tGMtZVkzWzbeH0HQZ83BaVEY38ArGEPDU1fh4b5NS0NmhhDqQvUCx2+gYqDpQ2dA3RXY6lJ+nMYwL88qcU4GV1OQ4XikVuTQfE2gR7qtjZbpY1sR+f3WKZ3u0TirjjgwV4sj7L66NV1qs/dJJFJyTEBnkjAzqJYqqTYcnTLHRR90qmsdCFJr/IdmjH2A7d+XwPfwUqQ5Tkif3rWIj2weT69ou5B19pzMY9rFkKuEKZ4kh158jMrHGwu1ldQ3vo3hA+L65D+DDiGhU+7rAZtei+GSL99bU1zEnQqcOwtVfQ8sP3bra1xS0/gfJLSSP2I7MhmTLYs53H6Sf6r/5pen134lSM5X0zpKXGmkO+YZHLqpnKGDL++n2aGkUzohxQRGow9zcXg1qX3kpaTBHLMUUcsaSI9J2mpcxg/f8EM8hDzGDh4QW1ikv4NoX5BED/Egsa+iYuzE6ca9fUVlWroXtZDs06/s735SVBVpnpO/OmuV/PFQV8vmb5ik2AexOSlWGAjXieSC2kRKwmeoDFvr2BhrdYEVrqDHTluUqQQXzAODs90qdvojqKubBR7qxWbV0lVp1dsOos/hvVeam0w1yNeH1rlEL6nwn8seABQBg58U4bhpATj4kRNipkRIhjR7GV5+9vxLVa3oijZwQUqX3+D8q1tMQAbBicXGwA1tNfbpal7s/UM9QVTZvwfBZZGhCnSA4Tcx3s9zgLveVkeAG7kpSL766iKxJNdYtYFyrK2Jm8QCA25Jud2fqILjL5RGdVtpybmDYO46tniy1K+n/7auRiRs46klfUbRXRotsqXPfAhLyajsEJI/aCoT7brO/oZG6r0NLOy3/qbKEV2ZraetYwn5o7UxOhx9/pD7H1SaLIhqWV07fD8BxAu8hK0E1Ph/+4pDtgOcxyE0ljyFWkPy8RHLsGqDke2HkKhD7K8n6kUD8wnoras2G51YbP3UKVyaoIuYjjLCC1CmAVhALbgwKhQApkNw26/6AvMTIKJctnxLiBlvqq0RgYPvr1zeGKqH1b7IBTCQNjVXERvh5o2J6n/waW/eqnzkPeH6+yWrV1nw8AadxvXER+yqPV7YMI93TTuId1Hm1UmD43mV7eWHLXWMCnZafZiQHmvg0jO1FLI6oVRh506yA2XJF32MGGqbsLpmdXDZnuW1e+ZbfVSR6blKQASCREg7QfYUseTp3GuvwGWpd354v89p8avgmfe3uoJPCLNctQvYrA9coVuXKUedCNpqbp0bKW6GgMLuhgAvQuQBasSXxGaxJ8SjIil3CxwYblK6ZvjFLq35Pv9YBC19us3WpGoRw3v/icKgMRJxCVRMCiw2yMQGdNI4tnZHDDmnQIea4yZVnPD3+a5u7gK4jWFH1BvM7NOzxbO3sfXTnWJPM2W721Gs+Lk5D3Ht1OzKnpoqXhauocuv9VC6ecAnQLJ7r9G6xMhk6cla9QCx88hli7LYowbHn2cHXzSz5x/mf4x7tjFWH265Zbv/OlR5gqGVlHOS6u8ric6s5h9OjuJfcmKXRAJn8Lt+mJxA56IsGjGnXf2JSRrjeWwzdKW3zEAHOqYFBOE1cpchbCaT273JIu2mqLcA+22UEx4roFZdSO3XxJBg6foKE4bSkTctEkpLJtcArZQJFZwNKkSxmtvkVDv/k7kzUqC3zhLfLwT648f/2Faf63QRyh1pt2URGKhiTYX26+D0kkElR/2fch0VRp8WgPahi6JpZIAV8LsbWDuTSFUg4f9Rx1VpYKi4pWWGoUThsRs6TnSnyBRFHcfQVy7i8syJkvWhCijsZsZ6jEWCrFUmM21Hbmf4Mq5GSuQuHwbL5XxBHd7L3XC4++0aWJ8L1i5HdMBJTg5hGQcmroxis4wDNsA018qFQklKIlwS7fN56M3LN+2XJUV1X7EyLlbhyOyE/FaN5atmTebLYHXKAub3T/s8sbebJP2Zc30OXjgET6+o1ZxmIBPPPHvvrlPHW9YCd9DVEglClLzyDi8wQ6Bbqxso4ScclGkmv+U4x7XEdRRap9tzPlfOHW6IkY5/dW2aGSG0PuF7jwvEJTihu6sSz9xvkEZNTQxRtoOIL+bp4/laWKT8mrMpwbGexuLM2MCw8Jj0s9UnKiWt3Q2NhQr1Y3NGmbdY01FXmZSXGxioT0w0WVNQ1anbYJ/rG+UaNtUp8szk5RxsTE7z+Yf6yqAf7fGmjd6Heahqqyo+mq2Ji4pANHy06pNTqdthEeWd+k1TbWHM/PTFLExCakHSk+Uduka9Y1NajJE+tOFB3aHx8Tq0w5VFBRDS/TaeCf1A0arab+dOmRNOqJpafqmrTkiQ3UE7OS4mJgjbD+OvzEeqix5ImFB5PjYmLIE+s16Ilk+fDEshxYY2x8cmZeeVW9trkZnkgtv/pYbkaCglk+9U9k+RX5mYlo+fTLtE31i5avSs0uqqxthHXA8uvU6GXqUyWHU5Sx1Jcx/8T+svj9WWQbm1nbWH40PQF9dGZu+WlmiXgdsMQDsERFYkZOyUnLddRW4o+OVaUdNu8wtR8n8RJj6B22XEcO2WH8MvrMyA5X5B1IJC8rJfvRBCGKnthYe7wgK0kRjT/6OBUgZB34o1OV+MvyyBM11BNhh8vJDqMnouXryPKb6CeS5UPINbKWr8EhF/8/H3JwMKnZhcdrLA6GWn4svMxir5pI7OAzS87KI2emZdZRXX40A51ZIglUtA51bU1NXUOTpqmh5kR5UX5efmFJ+fHKyuMV5aUlJSWlZeXH4K+8rOz/AbOrYRw='

pt = list(zlib.decompress(base64.b64decode(pt)))
print(f'{len(pt) = }')

len(pt) = 18109


This decodes well so far. Let's have a look at this in the all-purpose Audacity (as unsigned 8-bit values):

![scantest1](scantest1.png)

Well, there's clearly something interesting going on in there! There's a bunch of very well-formed waveforms, perhaps that gives us some insight on the path traversed through the binary. Those peaks look particularly intriguing though, let's hone in on those.

In [2]:
peak_inds = [i for i in range(1,len(pt)) if pt[i] >= 0xF0 and pt[i-1] < 0xF0]
print(f'{len(peak_inds) = }')

len(peak_inds) = 256


There are exactly 256 peaks! Is this a coincidence? We confirm this over a few different power traces (not shown here), and are definitely convinced there are exactly 256 peaks each time. It goes without saying that each peak represents one iteration through our loop.

In that case, could we learn anything by taking timings between peaks? This only gives us 255 readings of course, so we'd have to guess the first or last value, but we gotta start somewhere.

In [3]:
diffs = [y-x for x,y in zip(peak_inds, peak_inds[1:])]
print(diffs)

[63, 55, 90, 54, 123, 56, 63, 124, 28, 123, 63, 90, 125, 30, 29, 53, 27, 28, 54, 89, 123, 125, 65, 63, 28, 54, 28, 54, 123, 64, 30, 53, 123, 124, 63, 125, 56, 124, 30, 54, 61, 55, 27, 28, 55, 54, 29, 27, 61, 124, 64, 63, 125, 56, 63, 55, 61, 63, 29, 89, 55, 28, 62, 55, 123, 29, 62, 124, 125, 90, 30, 89, 87, 125, 55, 123, 30, 55, 28, 123, 124, 124, 29, 62, 90, 91, 63, 62, 89, 91, 87, 30, 90, 88, 30, 89, 61, 63, 125, 63, 91, 125, 31, 122, 56, 27, 123, 63, 90, 92, 28, 61, 29, 54, 27, 55, 54, 28, 62, 125, 88, 30, 27, 124, 125, 64, 64, 124, 55, 61, 30, 89, 88, 30, 63, 28, 61, 91, 88, 63, 92, 88, 30, 54, 89, 61, 55, 54, 124, 125, 30, 28, 61, 124, 64, 89, 125, 56, 29, 27, 27, 28, 123, 29, 89, 56, 28, 54, 89, 122, 64, 29, 88, 92, 63, 55, 27, 61, 56, 27, 55, 54, 29, 61, 55, 53, 90, 62, 125, 125, 64, 63, 63, 89, 29, 62, 125, 92, 62, 55, 61, 63, 55, 62, 125, 56, 29, 89, 61, 125, 125, 63, 90, 125, 64, 63, 62, 29, 28, 62, 64, 29, 63, 63, 29, 27, 28, 62, 63, 64, 30, 28, 61, 63, 29, 62, 63, 30, 29, 6

Let's plot this on excel to see what it looks like. Matplotlib also works, but ehhh.

![scandiffs](scandiffs.png)

There appear to be four distinct groupings, centered roughly around multiples of 30. Or maybe five groupings, since the ones near 60 looks like they can just about split further into two groups. We can divide it by 20 to get our five groups (i.e. values in {1,2,3,4,6}).


In [4]:
diffs_discrete = [i//20 for i in diffs]
print(diffs_discrete)

[3, 2, 4, 2, 6, 2, 3, 6, 1, 6, 3, 4, 6, 1, 1, 2, 1, 1, 2, 4, 6, 6, 3, 3, 1, 2, 1, 2, 6, 3, 1, 2, 6, 6, 3, 6, 2, 6, 1, 2, 3, 2, 1, 1, 2, 2, 1, 1, 3, 6, 3, 3, 6, 2, 3, 2, 3, 3, 1, 4, 2, 1, 3, 2, 6, 1, 3, 6, 6, 4, 1, 4, 4, 6, 2, 6, 1, 2, 1, 6, 6, 6, 1, 3, 4, 4, 3, 3, 4, 4, 4, 1, 4, 4, 1, 4, 3, 3, 6, 3, 4, 6, 1, 6, 2, 1, 6, 3, 4, 4, 1, 3, 1, 2, 1, 2, 2, 1, 3, 6, 4, 1, 1, 6, 6, 3, 3, 6, 2, 3, 1, 4, 4, 1, 3, 1, 3, 4, 4, 3, 4, 4, 1, 2, 4, 3, 2, 2, 6, 6, 1, 1, 3, 6, 3, 4, 6, 2, 1, 1, 1, 1, 6, 1, 4, 2, 1, 2, 4, 6, 3, 1, 4, 4, 3, 2, 1, 3, 2, 1, 2, 2, 1, 3, 2, 2, 4, 3, 6, 6, 3, 3, 3, 4, 1, 3, 6, 4, 3, 2, 3, 3, 2, 3, 6, 2, 1, 4, 3, 6, 6, 3, 4, 6, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 1, 1, 1, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 1, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 3, 3]


Can we coalesce this with our reversed code earlier? Let's have a look at what the different timings could represent:

| Bit is set | `result -= n` | `b -= n` | Operations | Type |
| :-: | :-: | :-: | :-: | :-: |
| False | False | False | `+` | 1 |
| False | False | True | `+-` | 2 or 3 |
| True | False | False | `++` | 2 or 3 |
| True | False | True | `++-` | 4 |
| True | True | False | `+-+` | 4 |
| True | True | True | `+-+-` | 6 |

It looks like we have exactly six different paths, and most likely we cannot distinguish `++-` from `+-+` using the timings alone. However, it does look like `+-` and `++` have slightly different speeds, which would account for there being two different groups just above and below 60.

Since solving our flag is equivalent to discriminating the `Bit is set` from the above table, all we really need to do is identify whether `+-` is actually type 2 or 3. Also recall that we have exactly 255 readings (from taking diffs between 256 peaks), so we're missing one bit, which we don't know is at the front or back.

No matter, we only really have 8 possible combinations of things to brute force: (type 2 or 3) * (missing bit is front or back) * (missing bit is 0 or 1). Let's brute force them all!

In [5]:
from Crypto.Util.number import long_to_bytes
for bit_is_set in [{2,4,6}, {3,4,6}]:
    bits255 = [int(i in bit_is_set) for i in diffs_discrete[::-1]]
    for missing_position in [0, 255]:
        for missing_bit in [0, 1]:
            bits256 = bits255[:]
            bits256.insert(missing_position, missing_bit)
            print(long_to_bytes(int(''.join(map(str, bits256)),2)))

b'6\xb4\xb274\xb3\xb4:=\xb6\x19\x99\x9a5\xb4\xb73\xaf\xb9\x98\xb22\xbb\x9a<\x9a\xbe'
b'\x80\x00\x00\x00\x006\xb4\xb274\xb3\xb4:=\xb6\x19\x99\x9a5\xb4\xb73\xaf\xb9\x98\xb22\xbb\x9a<\x9a\xbe'
b'midnight{l334king_s1dew4y5|'
b'midnight{l334king_s1dew4y5}'
b'j&\x9b8\xd9\xff\x9b{\xfc\x82w\x14\x1f3?Z\xf9\xc0\xbc\xbf\xb7\xfb\x8b\xbdK_\x01/0\xf8\x1e\xd5'
b'\xea&\x9b8\xd9\xff\x9b{\xfc\x82w\x14\x1f3?Z\xf9\xc0\xbc\xbf\xb7\xfb\x8b\xbdK_\x01/0\xf8\x1e\xd5'
b'\xd4M6q\xb3\xff6\xf7\xf9\x04\xee(>f~\xb5\xf3\x81y\x7fo\xf7\x17z\x96\xbe\x02^a\xf0=\xaa'
b'\xd4M6q\xb3\xff6\xf7\xf9\x04\xee(>f~\xb5\xf3\x81y\x7fo\xf7\x17z\x96\xbe\x02^a\xf0=\xab'


Success! Our flag is `midnight{l334king_s1dew4y5}`. Additionally, we learn that type 2 is a set bit so must correspond to `++`, and also that the missing bit is the least significant bit of our flag (so we are missing the first value in the power trace).

---

# SCAndinavia 2 (crypto)

Challenge file is provided in [scandinavia2.elf](scandinavia2.elf).

Clearly we did the bare minimum in SCAndinavia, so surprise! There's a second part! We get a different binary, so let's see what's changed:

![scandiff](scandiff.png)

Essentially the first two parameters of `multiply()` have been swapped. As the string change suggests, this actually makes it a lot more difficult. Namely, the above procedure now only leaks `A`.

We're certainly going to need more than before. We know that `c = a * b % n`, but let's look at a more complete equation:

$$a b - k n = c,$$

where they must all be 256-bit integers. We can leak `a` from the above procedure, and we have `c` given to us outright. The flag we want is encoded in `b`. It seems unlikely that we will be able to leak the flag directly, so it follows that we want to leak `k` or `n` instead.

But wait! `k` is simply the number of times we've subtracted `n` from the full product of `a` times `b`, and this is certainly something we can keep track of. If we can know the exact path traversed through the binary, then we know exactly when `n` (or some multiple thereof) is subtracted.

Does this give us `b` directly? If we know `k` we can take the equation mod `k` to get $b \equiv c a^{-1} \pmod{k}.$ We can certainly solve for `b` mod `k`, but this gives us an infinitude of answers. Sure, we know that `b` must be 256-bit, but if `k` is small enough (which it will be if `b` is small), then the number of possible answers is still huge.

How about if we leak `n` instead? The equation then reduces to $n \equiv c k^{-1} \pmod{a}$. This works a lot better since `a` is large (and we can verify this quickly since we leak `a` before anything else). This means that there are really only a handful of values to consider for `n`. Indeed, it becomes a whole lot smaller once we notice that we must have $a < n < 2^{256}$.

Anyway, with that out of the way, let's see how we can actually leak `k`. We can introduce an auxiliarly variable `k_b` to keep track of how many times `n` was subtracted from `b`. Then the parts that matter are:
* `result += b` ($k = k + k_b$)
* `result -= n` ($k = k + 1$)
* `b += b` ($k_b = 2k_b$)
* `b -= n` ($k_b = k_b + 1$)

This does mean that we will to distinguish the two type 4s (`++-` and `+-+`); maybe call one of them type 5 instead. But we cannot distinguish this from the times between peaks. Oh well, let's just brute-force them all. We need to find a power trace with as few type 4s as possible, and after a few queries we get one with only nine of them! (There's around 20-30 on an average power trace.)

We skip the part where we analyse lots of queries to get one we want, and here we assume we have a good power trace. We start off similarly as before.

In [6]:
c = 0x608d616c4d9f759f7d498d6e7b4429835eace71411278a34165c76debaa9e84d
pt = 'eJzFe2dUlGfX7u9z1jrr/PrW+j4FFLC3xKgxMTFFusD0AtOZofc+DCAiVUTp9kIZ6szQhmEQECnSS4yJURM1tsTExBh7QXTOvu/neaaAvjnre398/koW65n7ue/72ntf17X3c7CiouLgwUOHDpYdyE2NFHhsdv6P//O//vf//Q975xWrVjgvtfuv//zP/1rssMTRcekSB3s7+Af/vWzl2g82f/qlO10Qqio81do3OjHab9TVVp6sVDc2t3d0tLdomxoaGpq0upaWlmadVgP/tFod+qcl/1n/N/qf5lZ915mBc8NDZ/QNx0vys7NyCoorjhw7duRgWXFRUXHZwcOHK4pyVJFSLm3nThrbTywLCJCKhOKghIK64ZtPZh//ZCwM+GKlg9NGd2GUUpUYLuXzpImlnVeemkz3J07GuK9ebLf6S164Mj0lWs5n8YP3qMfvmUyvbxj3Crc6LnLc5CWNS01LDBWxfGn8oKSc4oqyvalh/j5uLu6+XLEiOEghFUuCk4t00/dem55cbsvx3+pk57xlpyxWpUoIFbJ86X74qVLrp4Kop5pn7s2Znv2ozxN+6mznvHmnFD+F1uIpErKKysv2pUeKaO4ubt4ckRw/JQ6ML6gf+2XWNHu7rzT461X2jh+4CiKVqUkRUi6DI0893n97zmT6e7oqfud6B4e1X3BCk1KVUXIenSGI3td8/v4b098XGtNZmxztV3xKV8SrUuKCBCw6WxKZXlBWfiAzVsr02OHixRQEBAUHBkgkiti8mqEbT+eeXO0qDNi+3M5xo7soOiU1KVzCoTP9g5Nzi8tL8pTBvJ0uO9xpPElgcJBcKpaFpx/qvPhgbvbeRGWC9wYH+9Wfs0OSUlNiFH4MGlsavauwrHx/ZoyU4b7D1YtFLCUWw1LqoZvP37640Vsc+OUKu6UfuAmjUtDGODQ6Tx6/5wA6jgghzW0HcRzBCplYEpRY2DRxd9b07Kphr3jbMjunjzwlMarUxDAxOsTAhOyiirKCtAiBr5uLmw91iPDU/qbJ316bnl837pN+vhye8hDDtuApti+NK49DaxXuihLTPeDo2cIAeEoGbxiXpz53+6Vp7tfBQ+GuaxyWrt/hF56cmhwp4zLYMuXhMzdmTabH39Ym+36wxGH156zgRJUyRsFn0P0i8pum4aYfX9Rl8rY42S/f6hMQp1LFhwjh6MURaXtLyw9kxQew4Og9Gf6yoOCgAIlYHpNdefba47lnN3pLg75eab+UumgZnyuOK2r/4THc80x1vNfaxYtXbeeEJaerYDG6L0sSlVFYXoHu0vaA5TG51YM3nr19eQshB35wgwuJHA6N2vS+XYA3dMBs6oADE/Y1jv/6yvT8WlfhgqPiKeIJlMK1WFBqfmrW9OLn7gNWuEkMl7B9fTkBcZn7y8sLM6IlDDjgnegNiQOOza0ZuvnCNHf33JEItzX2S9Z/zScPmObDFEWkFZRVFGUnBnJ3urp60vlSRRBgVITh9sNDOPqL2t3cLYDsT3zlCNnBAoaPOZDgNhkeru7oNgOpxaoHrj+efXy1a7/8ixX2OENgvPnxA5QHu68+N735behgyFfLF9mvdxFEqdKVkTIeRxhd2PLdI1jsQr3Sd73d4hXbWCFJaarYQH82V552YvAOhN/dgbLAL5YvWrLBVRitSkuG0GTyQ3Pqp/6ExPJzZz4klsVOm3bK4lJTIUUw4R0DE0mcArqtcWo5xtP7/3vHGOm+1mHJOuIYowJ4DLY0+VDP9Vcm05ML9Sm0D5fYr/qMGZyIYpNP90YZLruovBReRESHw/Lh4MOSIujnq4duPJl9dMVQACiwp1AAeYDBEoam5JWUF+ckKjieLjs86H5SQLBcIg6I2nOi98dHr5/f6q8Ic1kN17nDL0KJrpPHYPDxvRBp0XWHmw9KpsGwlDQkpbT1m3uzc39925jO3rTUbsUnNEVCqgpuk0ljicJTIViKshPkbE8XCBY/MljQUmeuPn47e/fc0Sj3tfZL1n7FC0uGzBfAZ/ODM2vGfjeZ3tzu2S/d5rxo6UYPSBFpKEX4+HJRYqkoB3iI6O442OVUsO+tG7nz0vTqdl8ZyrNLLXmWyQ3MqBy++9Zkujd8KGzHKjvYmD/emJTj68MUR1pw6oZwKiFwKg1Lq+j4/m+T6emlliy/rc72yz8m0kAwpAFeyB71KNz0s5868kWoGEDhiUHFQMymWw44kOPl4oIOOJA44MjM4z1XHr5+cWfwcCSKl3VfoYuGfO/HFUYV6L59APf8fWMq/QP7Rcs/ZQQDTuOCAKcBqmP9t16bTL8PVYR8tWKRA4J3Shq8PuA0JKtu4g+Tae5mV4H4U6fFjh95QVGCcBexyFiqQEVJuLAojf7yyvTyZm+x4ssV9o4fErmbwKksdjfgdP/uGCguFE4DZSgh5VQN3IAw+33kWLTHOsApXFoSgVOWJKni9NWXJtOzi01pzI+W2q/cxghMUKXEBvox6PywnPoJSN9Pr7TnCD5xtl+2haqc6LDCVPklAJFEBYYIHUOEQuNP/yYa7f+n0Qi5O+grq2IAiPNly2J2F8IBo+rt6eK6kymQBRIHHJ1d2X8dDvje2IlYz3XAB77kIj4QLedBPhWit68ozk0O4nmbcSoHnIamluu/A+g8vdKW7Q84XfaxN6IzUK6Y6BWJ6ghZh+np6ubNQospiMVO9V19NPv0594Skpj4o1eMlPnxZYllmHH9MXo0wmXlIru1X/lFpKSj8+AIIvK159FiP2jSmR8CTj+hByWmpQItYXMCUo703YSaeu/codCvVyx2gDCLxDjlYFSh7Lc/I1rMcIdNs4SozMkIHnHu1gt0VGTguvgTR8Vj8UMIevf2lzNF8s+XLV76IZlPxRwGBHXVyF0TXg0H9QYXFNT4iMmgLn93UBtQUD+51LKH/7GltgNA4LAUBG3ZhVI0HBZbQBYfBP3+a49mn1zrLgqE+4SAMbM4ligq62jrwOQ3U+dOt9RVVdY06PTGrq7O9hYd0OXm1vaOrr7R73+B43z7x3RdGmfrcsfVn/oGxCiVscFCNoMri997qmNo6pvJwa7muuoqdUOzvrPLaADyjR9va9fr29uaG6sP70uLlLC8PTx9WH4SuUIuFQkE4sCYjOLq9rOj46P9XS0N6praBl1bR2cnUHdEy2FxQ8/Q+Z//fgMX9mPnfsXXax2Xf+TmH56oTAiHV2f4haSW1J0emZoaPWvQ1lbX1GlaOzqNnR1tzVqNVofe/XT/5E9/Qu6Z+2XoaMzOD52c1n/JCY5XJkHOYvPkqqN9N14hnByLdFtt57AW8pkyDXDCpfly5Mn7a4zDU9Pj/Z26upqa2qYWPfnToBRaqH011RzZvytayoZ9eTP9JAEKuUwk8BcpotOLqtr6YF8Dp1sba/G+DJ2dBn0r8Xhbx+mB6Wt/vUEvNngkeueHzs4bviJfjM+AUA9ToRScmxzM93Fz9aDxxDhYxNJQVWnbt/ffWoJlizlYgPuH7ipv7Bmdmho5o9fU1tTUE8eBFsXHQd1F1cH8lDAhw8vDy5ctkMoVAeguJMHxe8rrDANj4yP9xpbGWnVdU3O7odPQ0YbuAh2moXtw5voDqENvfz13LNZ7I7zzl+wg8p19maLIzINN3SOTE8NnOrR1anW9FlbHW9ZZnxisvlcVJmLu9PC0Wl0cFLenrBavfrazuR6Q0IhWJ5CAYdjZO3zh1iNY/cEF7R7BZysdV368UxKVrIwLFXOYHElMzrHW/smZqXPdrQ01VQDidgKFGMRw3sYzI9/dfgLxdn+mYRfvk+WOqz7xkQGI4xCI2cKwlLzi8qKcpCAge1Dy+GTJk0VkHOm69Pfc7G9jJ+OQxFlDSJxo4Nk0jjQ6A2mL3TESRH13EtQXp8/82mHICc9/7i4CrmdnqU+gLfxC0svgkqaJS6quqdfCyxktyNAbuklozN46WxHhscHJ+YOveSEJysTIAD6LK1cdOfMzYPavyVOxnmvt7dd8wUW5HuVYX5Y0Pv9U+8DE1PhAV3N9TTU6Q4xZMpTRDejbWzTqo4BZGYfArBhhVgyYlUelH6hsPTMyPjrY3dZEYdbqzTq6zk5c/h3WNj04j85whRMkAllMcnJssAApz2BVkdowOD6BbrBBDfhp0QN+9G3NJH7Q81f+mEV3MFWTwtqy3GntZ3R5bHJyTKA/05fhH5peWmccmpgY7jPo6hc+39YO/1p1DafK85QhAjqgl8bB+JEI/QWSkITsg/XGwbFxm6c7SfS2tOop/Ly9/03jbr9PVziu2uotjQb8hIg4TG4AzmLTKIvp6qoAPziL4USCTw4OTk9iN5zErr8tdjv6EXaNOItZYZeInJ5z52+gLPb4Uvte2ZerHZdv8hBGJCkTwqR8niT2QOt3oCcefVunpG2wB4rPDKYoviLtxABJ8YPeS/GNe4WfOAHFx9QJWAlQfAyy7hGUCTpQJsCJkUg/GttYnr3VVxHuvh5AtoMXikHGo9NY4ojUvSU4HqyKNVANglQ+QOJHh8XP8q1I/KgI8cNHwqKcEBbuLu4+FsKWUNAwBnTz9Z3+8pAd1kUS8YmAhIJK/cD45NhAl/n0jObTa8H33gaoPXZgd0wAx8fDw5vJt6A2Mm3/qRZA7dhgd3tTnbq2QYsqiAGQYoXa316YTK/uDB2L89no7LTuCxbKWtEKfxaTJ4vJ2FdaRqoacwxLJMAx60ZuPXv9+Io+Hyv9TZTSZ9OZAoIh5yYFcr2QBKHSRUDE7qOnLz+ce3V35HgMyYCI6OQD6QtOLW3oGZuZGe3TN6mrqus0bQZjF5WdW4jgbNXVnyrLTQ7xJ/AtpPAtDUnMOdTQNTRO4rsGsrMVvqn8CCIdqEF7vuQLwNhmT1FkkjI+DJkngjBcGsxZBwHCiEqDzmbxhsqKPGUoURqI4JIKcWnIqqjvxMGFkzNavNNoBfDOnnPf3gSZCoKis1D+9Roo0+4CBHAo03QGP0hVVGs8N4nLdF11tVUtNdclFF2VFea6xHlXXaLyilVdIivDd7eBn5ieXm7Pl8LOV5A7h8pAo3MDYiFLlxXujpWxvFzddjL9KboalXWy79pT09s/sW+0xGHNdnZIIr4sJksUX2K48sxkenG5eTd3syMm4ti+8mci8VQzAvLj+dXOAslnoE43eYI6VSWEiTl0liBUicUThobLfGj8/frlL0DSPdYCNAiSjqBB55F2SHqE0Jfwm5AekBFW39Rvr948vNiM6J7dMlLDhYi4XJkSpPVLiuUutl/7NbBcYC8yLg0kvqqkvnt0ZmbsrEFjJkYG6rpb8XW/I5/JyXyWSdViY/O78pkN1qxPHB0B2k3m/jKCswMNRTGFaKhIIJSFpRScaD4zMjE+1NMOJUa9oPgZ+8Yv/fYSlZhvcIlxNJcYIZvlH5HXgFLem9u9RfLty+2cNnoAn1bhmOTIIVUiM/LBVGWcF6g7QnwgdYfMnKisI7ozo5MT53raNcBNGrSteoOhox1xE0gx5tpSWZ5niT2ZgqotKPaMQ/NqC6pM5Ftbasv5pkz/bStRbZFQtYUjjc093jYwOTM5dLqlvrqqph64idHMTVohQ/WN/XD3OXDB38Yqkxibljmu+ZyhiFMmxwQJuX5B6Sf6b70yzd3pK1FsX7ZoyQduRAkALsHwD999SHd24pvpkZ42YD3VcJwGfJzol81Rra09XrwnXsHz9fTYyeCJZChzCv2FsvCUvce03efGx4Z69BqKsEFOILbV3AqXMXbpLmRO0x+TahV7y3LHNdugXuP34nAkCSWGy0+wM5VC+8DBbuU27OxA2WKyxAllxp+eo7/VKX3hb6s+YxGujx+TKYwt1l+CaH1xBUeW3fJPaHIishiobqWV1nWdM3OAeg1x0q0WQj8wfRWRI9NvIycSaB8tc17/BSswDjgo+m0EkMnfZqHMttqyYxEIlYCEfdWdIzPfTAx0amurqtSNLR1G6xLfTmVAlIQgJN5Djt8XEpABbzyEgvroYmueZLtVSEhAYwkjdh/UnBmbnhrubW9UY9JnsE6/gCBIoOjxJ5fb96KIQgwhEjEEbH2Hppc1dCNi39dhCWd8U2Q5xzLpLZF/FZB/l33kJggn8y8EZOzuwrKyd1h3cYR4NXu0Fp1Po3ECYs2G3zsk70vTLFXNCemKhDKNxg1UFtV2gW4aQ7rJJtdjHtZOHDPKPKnhYjLzmPVgUGxmqbqjf5TSIAuZFMVb5ummxEisQYShytzi8uK8lFB/X3dXD18uMJBAOe5+HNAiT/wltjZXIEeRSh2+oJtB6xmGJqbG+o3NC2orpVxIBiJ7LwMhebOV8LFovav3QYQ+u9pVFOSyznHZRle/sERlYoSMx2ShytzUC9AYAWjUkLQAQ4OIYT0ol+/vQKC9/RPkN5eQ3zJKfuPk0jowAckFCZ9qNQgfg9FG+PSNXfz1GVLvUxDEH8Pj22gBEMSxRCcGdFPL2fHpyXM9bQBMNeL8Roo9NSNgAq5xqn98qS1PvH0VgetkKK4SLlsQmd80cx+o1bWOXP+tjoudNxM2M1KjDEF45iFd37g15jExazEnpw6EhLqTpdkJgXwaJCc6V4Rzrkgkj846eebqE9PcvfFT8ba1me7LlgJZ1w9OTo8PgsyprlZjmWO1Z+K6sDSPwtLc5z3SvM1KmrdbZE7/5I/IMzD9MVGtZG4GmQLpOBYII5YpgrD00nqUokYWpCgC4Mhz+PGP11jm4BMnZU5STKCAxeCHZtWM3HllenYN94JIaxsBkcNgiaOzjxFWTDeqFLZeiq4FWTFjF3+By3xzb9L2MoNFPL4CScMXuI0Q+vVKi38lg7CkyOf0KGQQdXX1PEKA7hkFFiTWFz/3lIS4Akg/dOGTIAWCokgurDYMTRIRUoPSHxHVpMtDaLt7oA3f/j52KpG+ydlx7XYm5OVkdGg0JrKUIS5zlSF+Zj8DWJFQIFLE7C6tJVWjzvzL+DKttBPII9PL693Fwa7rnMjoSQARRKfhVmJp2YGsBAVyyrwYfkSbRiwL33XYeAme+/vbBsLR/ZQemAD1CdUZhjByz2Fd3xiCPeYhIBoMpGjQmCMeuDhgMzHIj8CmUEZx8dCkvMNNp4fGxs4hpwPUlYUP4HSs7zo7SQrd6dpUztYVTms+owUAi4kJEjB96XxFPPCj0n0ZMVKWl5u7N8vf7GkC6n98OPv0Rl95qMtqh6UbdvhFoIZUgD9fSpbbP8dPRLmtWmy35kteeAruInP8w/OaZv4ymZ5fbs5gb3RYtGwrLTAxLTU+WMDmyJSHe5Fd8CciiatsSKIvR5F8QI0srrGzncQ2LO4ZvlTD6X7zpVYm0YGYrP2cqaAulSWO2nNY2zs6NXGOInOWvKejcgdWBc+vGknzzp2qSkzQtsdxWwA5GV4Wt5gkbNEEYSOIIlJ1rR2Gzo52m9/Geenl9dMoqTpZGYM0SBEx2Ue0PcMTJLfBz1NmFCJ8RCH5C/jikysd++RfWSsWSMpQrw9pz4zPQO5qg6RsVa+ppIwZnwm7UVm2bhSPh/xndFd/TVbGea61s1uNPJo0uCuQ1shXVBtHpmfG+ztJLUR6dBrq0AemfvoTwPP6zuARW8vS/19blixJ1K59ZRUH9sQHsDxdXL2Q0kFtddS4OdH70xPTmz8mTuFG/ZrtnJCkd2ZTbBrprVFAiDNsGmW8zzTqG5kYI4sfKb+JdNhMyG+MIdNfM8g9hljYZokF5OAW13YOEaYRQa3bcSi1WJlGl+/NEs+nciCb2jzvH5pWUotMo5G+Tp3V860kh6WkZfWhgtR3CB1rugHSkqAb1pZnz9A3Pz8AlDz6oS1fioybzaRxI+FC6Q5JzikqK85ThvC9XV3cfXkSBTGREJZ2sOO7+7Ovfh+vTPDZgLvzIST/xTZAY+/4NzNjffrGmsqq2qZWZAN0zLcByNTj9e7UM06kHrMUoQiy3og8updIS4yiPLzMyRKyLMSBCyo7hqZmJgaN2KxvpAoMuV9scxGVE+7ax3PBXZ+yGIRwWiTXoMgC3BWot/cLhlKcwbDPhQQDagWnoRYb3Qeqqiq/tKIEdYcIAxYfJWp4pCLD6a3p0feaDI5ZiCPDicUUxuxvvQg54OVPbXt4WxxBHfvK41NRM4Th4/uvOvE1Q0BfUR8QKXHcB8T9ai4NKgN+EahU2Hn3pJx3kTQkpaRl5rcXr/4AGKKBlpWWgRZgYdEQeWWFu6MldOwEC+XBeKAlMGFfw+id53OPLrXlWoZukEmHqFvc3krD8Mz5qSHsOc4XJIgf6XGHodBCY8jLEGAaU9nWN2LdYcCdE/NlQIK7hhLc82u4bqKCTrFOBtMfhFZ99+g05gO1JB/oJJSjhjIIWrV1J4CczS+AIOOT849oToNyPNerRyis11grR8xZRzHpNP1FynjsticDZxVxOOLYA23fP0I9OO0u1kdLQAGiyqyKoyrzEajMU7gyAyElikqH2c7Tm72HZz92FASgxE14qUhoMXmBu04OIhfg/viJaJu7hT2LE0o7fwRV+vySzgpKWHl6A1yI3hnRaHS3ajTGZFf2XX04++R6T0mQ9QQUkHe2ODrnePvQ9Pnpc6ebce+sWW+0bn6Z46nwPUz0nVdozj5IOKBSAJysNMwNm7T80MQU1AlgMnjyOFB1pQjgZPeW0GeSIADd2J0Xb5782DHfugQmBufbfBb0AtkoIc0IPcXKO4zU+T7+oTUXE/4tJOEXc+gMjJvTw1iJYs5g02Ki6irika9u9JaFuq13dAYeGWrhkUm2PBKrQ2OHdbdw3oGx8IFJ33lgjZi6U93CFrz4NbT469v9hyI9P3By3vA1l2ydoK5aOO7V5yQH871xbEuorlpquf7CX29RVzXLb6uTeVQiRMAkPIn6ruHJSbxlNQl2g9kmaceCwUz4Wf8G4cdtKxv1tsAaWktaQ4H/v9bQMLaGaihriGqn6Ts6cIC/q8wIqTIzjssMKqsgcjo7DVbWEAQ4EiKmB+cbM/if4gAnuylsOkuE6fVC6WfrNkN2KckC6Yd8KTpPRNptkF2U+UeRLzWOs4vaKrvozNnle7z4wwvaTP9tKxxXmls5yJwPz6ho6hmdmrTugmLdOc9nL7fy2WU2PrttH6nd0oVqIXb+63O0+HdayuujFqchbr8HcXvUgifmFayrjmXYzIEaNsNMnC1DLZDBiamxAaOFgxnJd7ZStP/QuCMUrS0HMxsQc2jYYOxUEn3zcud125kKAGi0wp/J9AvPrR+/O2t6egUNhi6zRzUqJgXNAvH4ckJSzv1ytjTwi2WLHMz9Jw5sNTART5qqQv28XXe4+3IlyLSGQ0Sarqyuc3B8chSROhTllFq0aLpvbz7EpkK7Na2CHM7jiKIKtOdBzjy73JLJ3bR0MVR1RXwqyBkhmy2OhxQON//ovDrJe7293So06pmWQkwiYiI7NDk9QU0i6Kg+km1uObwvPVJim1tQPd1VXIUnEWw79pZ6SjmPpPPkNK9jH5G+//DJ42XZcWgOkppplYmF0jByXvDRd5pd88qOD90f9WW6hycnhntBp+BKqjc3zdHFIzlw9f4b4u4SaUDo1m3Hhme0wg8ZXuEZ5Q2nkWFKNt3nuRHI2G5r1dafLMtJCibAbsMmcw834qbSGbOxbXNRqLWCisG/apri0Q9dLdE0Nc4b/dBbN/y9FjRNFziqBquGLdU0BfKEmwwIJJSjyvQPSswuKivJV82bjpaGvmOEKp4YoaJzsAc8/F4PmGqLgFpIixCzdiKIEOakWCQJRQTwd9yLwfN0ljklFmhhIAJmj5+w4axEVMsCEeXx3gBuWog9wlKyFuJO7xLikxYhbhv/etwhe/K+DhkvMLmwqmMAwrX/3e1r5CH+gBsBDy80AQxWktked2WYfNQOI8YTcWYj5pRVSZEyri/wOVxzCefHHfNp5MjiLBGdUVKj7x9DU/tE0tPNm/jAljoA4OHF1lzJFzaWOlCvKIrIkAXO2vjERAaVqKcmUL3v98reLvDKeGz/8LxGZKeAqsjibV6yyPljHznKPyEChi+DoE/Wfmk72XEx+2ikXfXc7MRbOqF0riwGNSP3o2akp00zMjq78uz1Z1Dn0XDiWtS+Imf40FyHGM3LVxRlJyg4Xq6ungw0kYlNroiMI6evoMns8/Uqxsal9qu2MYISiCYMUxBd2Po9/O31dUOe8BOyGZJC4JWjSD85cBNNrZ0pQQN+1mP9TGFE5mFEHaaGe1obqlFXaeHJkgNhU7VpnI/fMRCmJzQmZAXMiYlaZpMViE6zgGEZ45CKRLLwXYcM39+fnb03pVbSNy51WPWZeT90LnJOukZmsHNSW4WdE6O5i0z1NHUNlf8w3XT2H1rYuAPzpXW6Ad6LJ82ABE6N9hkI3tuipwYqNFRjAlNPNFBhO7XDY9A4svj8E23941Pjg1Bd1LamM/azkMuPOc2ji83ZQitDCdEKWD4NLw8c1EA5jgtNGtRBn3eouH8ZnJBVUYfa9zYd9E6rDjq195c/d5eEuK53Wk56rOFI5EiTKrpQN+8Z6DUoYA6k8sVOKo1vNnD65nfn8fc0qAFBmAKvbg8ejfH+0NlpPTk1pvBjMflBKUW1XSPT07hRbXupGhxPlE/39KfO/YE71qLOlj/2EGV+XFF0gQ5RhScXNenMjQ6LkZJMRFPHAjZ3/jTnun+e5hT5C2ThKftOtJwZxX4WXJTaZpyEnCi88p5UzOZI4ks6LkGifYrGiDcusVuBTWcVPiq6X3BybklFKSpZvkTJkgcF4eZU0gHt1O9zphfXuwplNhIBwiloNzEn+ufIUfSRAvkVCGRXjq83E/TDEW0v0U8nbU6yr002IzDTQlNzf+HOtBVVFvP9FKrDvdeem+Z+HSgPth3MpjMEkASa+yfPTxNJwNK0NpsTSD9oao8XZcbJqdZyANlaDohIKTiu6xlGreV2jWUW0CwAzK7g3K/DJ+J9P3ImWU0yGsphcOWJ+6oMkELGB5BNVa22siRtphcO/eseIkUsLHWFmhe5cHuBwk2a39Iy5Am2OhEtLcKxYfpH7tV++zf6mz7H72PHxc5bvAPiUGXAX1WYp9XRMLW7D/EdD55WR8PUr+a3S4lJX8KzLS/Kipej1I76F4FBZGo/ilP7wwuNaaxNOOoCiahjMvygRE2hL3x+aM5ExpdZr6IvfET4C5+ihV/4RL/nCx+4bhBNIBf7JmamKcPbdkahvcNseD/8HuenpSu2eImjSFsA2Bj6kqPEFtp4oF0JpOneLJAmHfWm8jiKNElick+0DwJ/JAYj1A3zBiMIpaapPVaUGSvn+nh6eDPMbVfAmGrfieZeAmOWWRKEEp2VCYWV2gXNfJlIY/ADE7IOlJUWoPsi/UHqY6J/9+sCW1cRtzMi8PA7mgoiPiaa/5HGwnluxn9nnlvmxw9A2foZpCec9xbZo7ynSifHggiMFqSF2353IAlRFjfP/D479+A77LLa4bE+7KSy6FC7CqoMQJrxbGQ1RZrnFXTEbQ9k/Iv5PPNUKZVQm0lyeRpSAXKs3yxIqAIOV0bWnr9nahJB8Nmv3k4KPj+m+duihzPVCTvXob9xLJ8bACE/1tI3NjUxZN2XoBpIrYioDRFEbfZmb2mo23onS8OTC2ISbozyikgfWE5NYJfhCeznP3XkiUAuL6PkMv5cIVC5v9owiKm0VTUk3ANzOwTt9uWt/kNRXh84OSEVmYCqoYAnQuMr996Ynl5q3s3ZtGQRUr4JaSi/cHjy1GP9t2ZNb345S3k9IsLrYWOvBwjbJDUGVDOfsCH3AG7rH8eAeokxIFKCWuII2D+OI4uZ6/vPZq6AxRLFFXdcfkp+VIOrISMoEaohdvyFUVlHmvvGpycwk65RzzM/KY/I0oL1mtcHOYINKlK5ztO9VsH/GOerVTZ8isYNICZigIeziaFAKTGiJpCEJOUe0XQPk4q6xnrWxmx9EWLoMTA10eernFZthV9OSka/jD7nTCw4ib9FRlquTl3XCPtCJZmaZoXnx69AxON/z7/VFacnpeQeajw9OHSm9VRhWkxYeIwqu+jwKXV9Q0N9XW1tHfqAWVN3snxvRnJ8fFJ6zoFDp2qbtFpNA/yxvlHTVF91+ECWKiEuUbVnX8XJ2iadTtNIPKdpqDlanJOaEBev3L237FhNg1anbayvVaM/NdaeKMvflRQXl5SeW3ykql6j0zbBn2rrmzRNdZUH9+1WxsclpmbvP2hZrK5Ro2moPnIgG34xARYrP6EmFyN+seZYcU5aIiyWsbfUdrEmq8WKDlfixRrQYuj1Kw8VZqYQix06VaeBX2wAJoMXqzq8PyvFvFij1rIzWKwkl1gsv/RYdQP1+ngxfFZosRxYrG7+YnvgFxNTs2BndU3Wi8HO0DEmWB+jWm05xsS4hJTMgvLj6D205k2rjxPvkZyRX3LU9j3qyPdITINjPFnbqIX3IO6MOEa0GPxi2fGaBo1WA2dVS/5iaV56Ujy1M+28ne0CFCTvyoM7M/8J39mpigJ0Z8Sm4YR1tieMrzOrEN4DwclynUeLsqmdnZi3M+qEATvHaxrJ66wlsINeMS6OeA9b7JwisJOUlnOAeA+N7eHHxxLXSWyafA/zYvgXK+sAck0Afowd9IuZFBpPWkN/wTFqFxwj/sWSo+Q7qqv/HyfQq3M='

pt = list(zlib.decompress(base64.b64decode(pt)))
peak_inds = [i for i in range(1,len(pt)) if pt[i] >= 0xF0 and pt[i-1] < 0xF0]
assert len(peak_inds) == 256

diffs = [y-x for x,y in zip(peak_inds, peak_inds[1:])]
diffs_discrete = [i // 20 for i in diffs]
print(f'{diffs_discrete.count(4) = }')

a255 = [int(i in {2,4,6}) for i in diffs_discrete[::-1]]
a_lsb_unk = int(''.join(map(str, a255 + [0])),2)
print(f'{a_lsb_unk = } (up to least significant bit)')

diffs_discrete.count(4) = 9
a_lsb_unk = 45356713048829625797471681831212399989824916906554803590621606417526349971256 (up to least significant bit)


This just confirms where we have nine iterations of type 4 (`++-` or `+-+`). Let's call them types 4 and 5 respectively, and update our table accordingly:

| Bit is set | `result -= n` | `b -= n` | Operations | Type |
| :-: | :-: | :-: | :-: | :-: |
| False | False | False | `+` | 1 |
| False | False | True | `+-` | 3 |
| True | False | False | `++` | 2 |
| True | False | True | `++-` | 4 |
| True | True | False | `+-+` | 5 |
| True | True | True | `+-+-` | 6 |

First we need a powerset recipe. We use the one from [itertools-recipes](https://docs.python.org/3/library/itertools.html#itertools-recipes).

In [7]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

Now we have all the ingredients. The high-level recipe is as follows:
* Try all six possible "starting" values for the first missing "measurement"
* Try all $2^9=512$ possible combinations of changing type 4 to 5
* Evaluate $n \pmod{a}$ for this config, and keep adding $a$ to it until it exceeds $2^{256}$
* If an $n$ is found, calculate the corresponding flag. Print if flag is all ASCII.

In [8]:
type4_inds = [i for i,j in enumerate(diffs_discrete) if j==4]
print(type4_inds)

for missing_type in [1,2,3,4,5,6]:
    a = a_lsb_unk + (missing_type in {2,4,5,6})
    
    for change4to5 in powerset(type4_inds):
        
        types256 = diffs_discrete[:]
        for t in change4to5:
            types256[t] += 1
        
        types256.insert(0, missing_type)

        k, k_b = 0, 0
        for i in [1] + types256:
            if i in {2,4,5,6}: k += k_b
            if i in {5,6}: k += 1
            k_b *= 2
            if i in {3,4,6}: k_b += 1

        try:
            n = -c*pow(k,-1,a)%a
            while n < 2**256:
                b = (k*n+c)//a
                bs = long_to_bytes(b)
                if all(x < 128 for x in bs):
                    print(f'found solution using {change4to5}: {bs}')
                n += a
                        
        except:
            pass

[99, 123, 131, 184, 193, 198, 208, 218, 253]
found solution using (123, 193, 208): b'midnight{d4n1sh_i5_h4rd}'


And there we go! We found the flag: `midnight{d4n1sh_i5_h4rd}`.

---

# Post-CTF discussion

We spoke to @larsh after the hunt (this was my favourite challenge after all). The intended way to read the power trace was to actually look at the actual power pattern (e.g. how high it goes before it goes low again), so that you can actually distinguish `++-` from `+-+`. Presumably you can read all 256 iterations this way, I haven't really tried it.

Additionally, the intended solution is also to use multiple (at least 2) queries: then you have the flag modulo different values of `k`, and can CRT them into the final answer. I think this is less necessary if `a` and `n` are both near the maximum of $2^{256}$, but is definitely a good trick to remember either way.